In [1]:
using Ai4EComponentLib
using Ai4EComponentLib.CompressedAirSystem
using ModelingToolkit, DifferentialEquations
using Plots

In [2]:
using Ai4EComponentLib
using Ai4EComponentLib.CompressedAirSystem
using ModelingToolkit, DifferentialEquations

n0 = 4000
h_polCoff = [-91.7802, 1058.2670, 3213.1520]
etaCoff = [-0.0181, 0.2880, -0.2557]
surgeCoff = [-2.950e-7, 4.8009, -5.1678]
chokeCoff = [1.1054e-6, 8.6274, 20.7626]

f=0.05                  # Friction resistance coefficient
n = 15                  # Number of nodes
D = 0.4                 # Pipe diameter
L = 20                  # length
R = 3000                # Resistance coefficient
T = 300                 # Temperature
qm0 = 10 * ones(n)      # Initial mass flow rate
p0 = 10e5 * ones(n)     # initial pressure

n2 = 40                 # Number of nodes
D2 = 0.4                # Pipe diameter
L2 = 200                # length
R2 = 3000               # Resistance coefficient
T2 = 300                # Temperature
qm02 = 10 * ones(n2)    # Initial mass flow rate
p02 = range(10e5,8e5,length=n2)  #initial pressure

inletBoundary = Dict(
    "p" => 1.0e5,
    "T" => 300,
)

outletBoundary = Dict(
    "T" => 300,
    "p" => 4e5,
)

Dict{String, Real} with 2 entries:
  "T" => 300
  "p" => 400000.0

In [3]:
@named inletSource = Source(boundary=inletBoundary)
@named outletSource = Source(boundary=outletBoundary)

┌ Warning: source contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40
┌ Warning: source contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40


Model outletSource with 5 (6) equations
States (6):
  source₊T(t) [defaults to 300]
  source₊p(t) [defaults to 101300.0]
  source₊qm(t) [defaults to 0]
  source₊ρ(t) [defaults to 1.2]
⋮
Parameters (0):

In [4]:
@named transPipe1 = TransitionPipe(n=n, D=D, L=L, T=T, p0=p0, qm0=qm0, f=f)
@named transPipe2 = TransitionPipe(n=n, D=D, L=L, T=T, p0=p0, qm0=qm0, f=f)

┌ Warning: inlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40
┌ Warning: outlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40


┌ Warning: inlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40
┌ Warning: outlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40


Model transPipe2 with 39 (41) equations
States (43):
  (p(t))[2] [defaults to 1.0e6]
  (p(t))[3] [defaults to 1.0e6]
  (p(t))[4] [defaults to 1.0e6]
  (p(t))[5] [defaults to 1.0e6]
⋮
Parameters (4):
  dx [defaults to 1.33333]
  c1 [defaults to 6.85425e5]
  c2 [defaults to 42839.0]
  A [defaults to 0.125664]

In [7]:
eqs = [
    connect(inletSource.source, transPipe1.inlet),
    connect(transPipe1.outlet, transPipe2.inlet),
    connect(transPipe2.outlet, outletSource.source)
]

3-element Vector{Equation}:
 connect(inletSource.source, transPipe1.inlet)
 connect(transPipe1.outlet, transPipe2.inlet)
 connect(transPipe2.outlet, outletSource.source)

In [8]:
@named eq_model = ODESystem(eqs, t)
@named model = compose(
    eq_model,
    inletSource,
    outletSource,
    transPipe1,
    transPipe2
)

sys = structural_simplify(model)
prob = ODEProblem(sys, [], (0, 3))
sol = solve(prob, Rodas4())

ModelingToolkit.ExtraVariablesSystemException: ExtraVariablesSystemException: The system is unbalanced. There are 98 highest order derivative variables and 97 equations.
More variables than equations, here are the potential extra variable(s):
 (transPipe1₊p(t))[2]
 (transPipe1₊p(t))[3]
 (transPipe1₊p(t))[4]
 (transPipe1₊p(t))[5]
 (transPipe1₊p(t))[6]
 (transPipe1₊p(t))[7]
 (transPipe1₊p(t))[8]
 (transPipe1₊p(t))[9]
 (transPipe1₊p(t))[10]
 (transPipe1₊p(t))[11]
 (transPipe1₊p(t))[12]
 (transPipe1₊p(t))[13]
 (transPipe1₊p(t))[14]
 (transPipe1₊p(t))[15]
 (transPipe1₊qm(t))[2]
 (transPipe1₊qm(t))[3]
 (transPipe1₊qm(t))[4]
 (transPipe1₊qm(t))[5]
 (transPipe1₊qm(t))[6]
 (transPipe1₊qm(t))[7]
 (transPipe1₊qm(t))[8]
 (transPipe1₊qm(t))[9]
 (transPipe1₊qm(t))[10]
 (transPipe1₊qm(t))[11]
 (transPipe1₊qm(t))[12]
 (transPipe1₊qm(t))[13]
 (transPipe1₊qm(t))[14]
 (transPipe1₊qm(t))[1]
 (transPipe1₊qm(t))[15]
 (transPipe2₊p(t))[2]
 (transPipe2₊p(t))[3]
 (transPipe2₊p(t))[4]
 (transPipe2₊p(t))[5]
 (tr